In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('iris.csv')

In [3]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [5]:
df.isna().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [6]:
x=df.drop(['species'],axis=1)
y=df['species']

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [8]:
num_pipe=Pipeline(steps=[('impute',SimpleImputer(strategy='median')),('scaler',StandardScaler())])

In [9]:
x1=pd.DataFrame(num_pipe.fit_transform(x),columns=num_pipe.get_feature_names_out())

In [10]:
x1.head()

,sepal_length,sepal_width,petal_length,petal_width
0,-0.900681,1.019004,-1.340227,-1.315444
1,-1.143017,-0.131979,-1.340227,-1.315444
2,-1.385353,0.328414,-1.397064,-1.315444
3,-1.506521,0.098217,-1.283389,-1.315444
4,-1.021849,1.249201,-1.340227,-1.315444


In [11]:
le=LabelEncoder()

In [12]:
y1=le.fit_transform(y)

In [25]:
with open('pipe.pkl','wb') as file:
    pickle.dump(num_pipe,file)

In [26]:
with open('lb.pkl','wb') as file:
    pickle.dump(le,file)

In [13]:
from sklearn.model_selection import train_test_split,GridSearchCV

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x1,y1,test_size=0.1,random_state=21)

In [16]:
from sklearn.neighbors import KNeighborsClassifier

In [17]:
kn=KNeighborsClassifier()

In [18]:
grid={
    'n_neighbors':range(1,10)
}

In [19]:
gn=GridSearchCV(kn,param_grid=grid,cv=3)

In [20]:
gn.fit(x_train,y_train)

GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 10)})

In [21]:
gn.best_params_

{'n_neighbors': 7}

In [22]:
best_knn=gn.best_estimator_
best_knn

KNeighborsClassifier(n_neighbors=7)

In [23]:
import pickle

In [24]:
with open('bestknn.pkl','wb') as file:
    pickle.dump(best_knn,file)

In [28]:
y_pred=best_knn.predict(x_test)

In [29]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [30]:
con=confusion_matrix(y_pred,y_test)
print('confusion_matrix\n',con)
a=accuracy_score(y_pred,y_test)
print('accuracy_score\n',a)
claf=classification_report(y_pred,y_test)
print('classification_report\n',claf)

confusion_matrix
 [[7 0 0]
 [0 5 0]
 [0 0 3]]
accuracy_score
 1.0
classification_report
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         7
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         3

    accuracy                           1.00        15
   macro avg       1.00      1.00      1.00        15
weighted avg       1.00      1.00      1.00        15



In [35]:
def predict_point(pipe,l,model):
    sepal_length=float(input('Enter Sepal Length'))
    sepal_width=float(input('Enter Sepal Width'))
    petal_length=float(input('Enter Petal Length'))
    petal_width=float(input('Enter Petal Width'))

    x_new=pd.DataFrame([sepal_length,sepal_width,petal_length,petal_width]).T
    x_new.columns=['sepal_length','sepal_width','petal_length','petal_width']
    print('Dataframe before preprocessing \n', x_new)

    x_pre=pd.DataFrame(pipe.transform(x_new),columns=x_new.columns)
    print('Dataframe after preprocessing \n', x_pre)

    pred=model.predict(x_pre)
    lb=l.inverse_transform(pred)[0]

    prob =model.predict_proba(x_pre).max()
    print(f'The predicted class is {lb} with probability {prob}')

    return lb,prob


In [36]:
predict_point(num_pipe,le,best_knn)

Dataframe before preprocessing 
    sepal_length  sepal_width  petal_length  petal_width
0           1.0          9.0           8.0          7.0
Dataframe after preprocessing 
    sepal_length  sepal_width  petal_length  petal_width
0     -5.868564    13.679826      2.411044     7.635544
The predicted class is virginica with probability 0.5714285714285714


('virginica', 0.5714285714285714)